<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Cost Management for Azure with Python

## Azure Actions Covered

* Query actual usage and cost data
* Query forecasted usage and cost data
* Set up recurring cost export

In this lecture, we'll learn how to use Azure cost management library with Python.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. AZURE_SUBSCRIPTION_ID). We'll add some new imports as well.

In [11]:
import datetime

from azure.identity import AzureCliCredential
# New imports for cost management
from azure.mgmt.costmanagement import CostManagementClient
from azure.mgmt.costmanagement.models import (
    QueryDefinition, QueryDataset, QueryAggregation,
    QueryTimePeriod, GranularityType, ForecastDefinition,
    Export, FormatType, ExportDeliveryInfo, ExportDefinition,
    ExportSchedule, ExportRecurrencePeriod, ExportDeliveryDestination
)
from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_RESOURCE_GROUP, DEFAULT_LOCATION

Let's instantiate our credential and then use it to instantiate the `CostManagementClient()`.

In [2]:
credential = AzureCliCredential()
cm_client = CostManagementClient(credential)

We can return data on our usage of Azure using `query.usage()`. The scope can be the subscription, resource group, billing account, etc. For our purposes, we'll just focus on the subscription.

The parameters for this method are somewhat complicated, so let's walk through them.

* The `parameters` must be a `QueryDefinitiion` object. For more information, see [QueryDefinition class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.querydefinition?view=azure-python).
* The `QueryDefinition` object will take:
    * `type`- Type of cost data to return. Can be "Usage", "ActualCost", or "AmortizedCost"
    * `timeframe` - Time frame for pulling data for the query
    * `time_period` - Time period for pulling data if `timeframe` is `Custom`. This is a `QueryTimePeriod` object.
    * `dataset` - `QueryDataset` object with definition for data in the query.
* For more information on `QueryTimePeriod`, see the [QueryTimePeriod class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.querytimeperiod?view=azure-python).
    * `from_property` - Start date for pulling data
    * `to` - End date for pulling data
* For more information on `QueryDataset`, see the [QueryDataset class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.querydataset?view=azure-python). We'll use
    * `granularity` - Time period granularity for data to return
    * `aggregation` - Dictionary of string (alias for aggregated column) and a `QueryAggregation()` object.
* For more information on `QueryAggregation`, see the [QueryAggregation](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.queryaggregation?view=azure-python).
    * `name` - Name of column to aggregate. We'll use "PreTaxCost".
    * `function` - Aggregation function to use (e.g. "Avg", "Min", "Max", "Sum"
    
We'll write a query to return the daily total pre-tax cost for the month of April, 2023.

In [3]:
query = cm_client.query.usage(
    # Scope will be subscription
    scope=f'/subscriptions/{AZURE_SUBSCRIPTION_ID}',
    parameters=QueryDefinition(
        # We want to get usage data
        type="Usage",
        # Use a custom time frame
        timeframe="Custom",
        # April 1, 2023 to April 30, 2023
        time_period=QueryTimePeriod(
            from_property=datetime.datetime(2023,4,1),
            to=datetime.datetime(2023,4,30)
        ),
        # Data to return
        dataset=QueryDataset(
            # Data aggregated by day
            granularity="Daily",
            # Sum of the pre-tax cost, aliased to 'totalCost'
            aggregation={
                'totalCost': QueryAggregation(name='PreTaxCost', function='Sum')
            }
        )
    )
)

Datetime with no tzinfo will be considered UTC.
Datetime with no tzinfo will be considered UTC.


Now let's look at the returned data. It's a list of lists with the values in each index of:
* `totalCost` - Based on our query
* String representation of date
* Unit of measurement for returned data, in our case US Dollars (`USD`).

In [4]:
query.rows

[[0.5245836667460327, 20230407, 'USD'],
 [1.9279980235436547, 20230408, 'USD'],
 [1.927998067446201, 20230409, 'USD'],
 [1.9279980336019387, 20230410, 'USD'],
 [1.9279981990048278, 20230411, 'USD'],
 [1.9279980024212588, 20230412, 'USD'],
 [1.9279979933688034, 20230413, 'USD'],
 [1.9279979531356681, 20230414, 'USD'],
 [1.9279980496951927, 20230415, 'USD'],
 [1.9279979863280048, 20230416, 'USD'],
 [1.9516757935772657, 20230417, 'USD'],
 [1.632e-06, 20230420, 'USD'],
 [1.3184e-05, 20230421, 'USD'],
 [0.23043020450235638, 20230422, 'USD'],
 [2.344589652566969, 20230423, 'USD'],
 [2.343072923380461, 20230424, 'USD'],
 [1.9532765839262187, 20230425, 'USD'],
 [0.13868817204301076, 20230426, 'USD'],
 [0.832129032258065, 20230427, 'USD'],
 [0.5200806451612905, 20230428, 'USD']]

We can use the same type of query construction to return forecasts of usage instead of actual usage. The only difference is we need to use the `ForecastDefinition` class instead of the `QueryDefinition`. For more information, see the [ForecastDefinition class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.forecastdefinition?view=azure-python).

In [12]:
query = cm_client.forecast.usage(
    # Subscription scope
    scope=f'/subscriptions/{AZURE_SUBSCRIPTION_ID}',
    parameters=ForecastDefinition(
        # Get usage data
        type='Usage',
        # Custom time frame
        timeframe='Custom',
        # Last days in May 2023
        time_period=QueryTimePeriod(
            from_property=datetime.datetime(2023, 5, 22),
            to=datetime.datetime(2023, 5, 31)
        ),
        # Data to return
        dataset=QueryDataset(
            # Daily granularity
            granularity='Daily',
            # Sum the pre-tax cost and alias to 'totalCost'
            aggregation={
                'totalCost': QueryAggregation(name='PreTaxCost', function='Sum')
            }
        )
    )
)

Datetime with no tzinfo will be considered UTC.
Datetime with no tzinfo will be considered UTC.


Now we can look at the returned data.

In [13]:
query.rows

[[0.0, 20230522, 'Forecast', 'USD'],
 [0.0, 20230523, 'Forecast', 'USD'],
 [0.0, 20230524, 'Forecast', 'USD'],
 [0.0, 20230525, 'Forecast', 'USD'],
 [0.0, 20230526, 'Forecast', 'USD'],
 [0.0, 20230527, 'Forecast', 'USD'],
 [0.0, 20230528, 'Forecast', 'USD'],
 [0.0, 20230529, 'Forecast', 'USD'],
 [0.0, 20230530, 'Forecast', 'USD'],
 [0.0, 20230531, 'Forecast', 'USD']]

We can also automate some of our cost management or analysis with exports of our cost data. Let's see the exports we've got set up already. There might not be anything there.

In [5]:
exports = cm_client.exports.list(scope=f'/subscriptions/{AZURE_SUBSCRIPTION_ID}/')

In [8]:
exports.as_dict()

{'value': []}

Now let's create a new export. The process involves similar types of parameters to what we saw with the usage and forecast queries.

* `scope` - Scope for the export. Can be subscription, resource group, billing group, etc.
* `export_name` - Name for the cost export to be scheduled
* `parameters` - Parameters from an `Export` object.

The `Export` object has the following parameters:

* `format` - Format for the cost export, 'Csv' is the only option right now
* `delivery_info` - An `ExportDeliveryInfo` object. For the full list of parameters, see [ExportDeliveryInfo class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.exportdeliveryinfo?view=azure-python)
    * `destination` - An `ExportDeliveryDestination` object. For the full list of parameters, see [ExportDeliveryDestination class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.exportdeliverydestination?view=azure-python)
* `definition` - An `ExportDefinition` object. For the full list of parameters, see [ExportDefinition class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.exportdefinition?view=azure-python)
* `schedule` - An `ExportSchedule` object. For the full list of parameters, see [ExportSchedule class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.exportschedule?view=azure-python)

For the full list, see the [Export class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-costmanagement/azure.mgmt.costmanagement.models.export?view=azure-python).

In [12]:
new_export = cm_client.exports.create_or_update(
    scope=f'/subscriptions/{AZURE_SUBSCRIPTION_ID}/',
    export_name='bens-export',
    parameters=Export(
        format='Csv',
        delivery_info=ExportDeliveryInfo(
            destination=ExportDeliveryDestination(
                resource_id='<insert resource id>',
                container='bens-blob-container',
                root_folder_path=''
            )
        ),
        definition=ExportDefinition(
            type="ActualCost",
            timeframe="MonthToDate"
        ),
        schedule=ExportSchedule(
            status="Active",
            recurrence="Monthly",
            recurrence_period=ExportRecurrencePeriod(
                from_property=datetime.datetime(2023, 5, 23),
                to=datetime.datetime(2023, 12, 31)
            )
        )
    )
)

Datetime with no tzinfo will be considered UTC.
Datetime with no tzinfo will be considered UTC.


In [16]:
new_export.schedule.as_dict()

{'status': 'Active',
 'recurrence': 'Monthly',
 'recurrence_period': {'from_property': '2023-05-23T00:00:00.000Z',
  'to': '2023-12-31T00:00:00.000Z'}}